# Analyzing & Improving the NIHAO BH Feedback

In the NIHAO simulation code, the BH feedback calculation part is in ```src/smoothfcn.c```. 

## Goal

- Implement the new, realistic BH feedback scheme

## Problem

- I am not so familiar with the C language. I need to learn first.
 - Especially I don't know about
   - ```*```
   - ```#ifdef```
   - ```*p1```
   - ```->```
 - I am now taking the LinkedIn learning course.
 
 
- Also, I need to learn about the NIHAO simulation itself. I am not so well informed of the BH feedback mechanism. 
 - Now I am reading the previous NIHAO papers.
   - Wang et al. 2015
   - Blank et al. 2019

## Code Analysis

```void```: a function which does not return any value

```PARTICLE, int, NN, SMF```: types
```*p, nSmooth, *nnList, *smf```: arguments
: what is type ```PARTICLE, NN, SMF```?
: what is ```*```? (Pointer, what is a pointer?)

```#ifdef```: conditional compilation, in this case, ```STARFORM``` must be defined.

Find the equation part!
```c
    FLOAT ih2,r2,rs;
    double mdot, dmAvg, dm, dE, dtEff;
    FLOAT fNorm,fNorm_new,f2h2;
    int i,counter;
    FLOAT fbweight;
```
: define variables, the meaning?

### initTreeParticleBHFeedback

```c
void initTreeParticleBHFeedback(void *p1)
{
#ifdef STARFORM
    /* Convert energy and metals to non-specific quantities (not per mass)
     * to make it easier to divvy up BH energy.
     */

    if(TYPETest((PARTICLE *)p1, TYPE_GAS))  ((PARTICLE *)p1)->u *= ((PARTICLE *)p1)->fMass;
    if(TYPETest((PARTICLE *)p1, TYPE_GAS))  ((PARTICLE *)p1)->fNSN = 0.0;
    /* fNSN will hold the place of FB energy because uPred is needed for
     *  calculations. it should be zero anyways.  JMB 10/5/09  */

#endif
}
```


### initBHFeedback

```c
/* Cached Tree Active particles */
void initBHFeedback(void *p)
{
#ifdef STARFORM
    if (TYPEQueryTREEACTIVE((PARTICLE *) p)) ((PARTICLE *)p)->u = 0.0; /*added 6/10/08*/
    if (TYPEQueryTREEACTIVE((PARTICLE *) p)) ((PARTICLE *)p)->fNSN = 0.0;

    /* Heating due to accretion */
#endif
}
```

### combBHFeedback

```c
void combBHFeedback(void *p1,void *p2)
{
#ifdef STARFORM
    PARTICLE *pp1 = p1;
    PARTICLE *pp2 = p2;

    if (TYPEQueryTREEACTIVE(pp1)) {
        pp1->u += pp2->u; /*added 6/10/08 for BH blastwave FB*/
        pp1->fNSN += pp2->fNSN;  /* (this is uPred) JMB 10/5/09  */
        pp1->fTimeCoolIsOffUntil = max( (pp1)->fTimeCoolIsOffUntil,
                                       (pp2)->fTimeCoolIsOffUntil );
        pp1->fMassForm = max( (pp1)->fMassForm,
                             (pp2)->fMassForm ); /* propagate FB time JMB 2/24/10 */
    }
#endif
}
```

### BHFeedback

```c
void BHFeedback(PARTICLE *p,int nSmooth,NN *nnList,SMF *smf)
{
#ifdef STARFORM
    FLOAT ih2,r2,rs;
    double mdot, dmAvg, dm, dE, dtEff;
    FLOAT fNorm,fNorm_new,f2h2;
    int i,counter;
    FLOAT fbweight;

    dm = p->divv;

    if (dm <= 0.0) return;

    if(smf->dBHSinkFeedbackFactor != 0.0) {  /* allows FB to be turned off JMB 7/20/09 */
        dE = smf->dBHSinkFeedbackFactor*dm; /* dE based on actual mass eaten */

        dtEff = smf->dSinkCurrentDelta;/* *pow(0.5,p->iRung-smf->iSinkCurrentRung);*/
        dmAvg = mdot*dtEff;
        /*printf("BHSink %d:  Delta: %g Time: %g  dm: actual %g (pred %g) (avg %g) dE %g\n",p->iOrder,dtEff,smf->dTime,dm,p->curlv[1],dmAvg,dE); */
        //printf("BHSink %d:  Delta: %g Time: %.8f dm: %g dE %g\n",p->iOrder,dtEff,smf->dTime,dm,dE);

        /* Recalculate Normalization */
        ih2 = 4.0/BALL2(p);
        f2h2=BALL2(p);
        fNorm_new = 0.0;
        fNorm = 0.5*M_1_PI*sqrt(ih2)*ih2;

        for (i=0;i<nSmooth;++i) {
            r2 = nnList[i].fDist2*ih2;
            KERNEL(rs,r2);
            fNorm_new += rs;
            rs *= fNorm;
        }

        /* spread feedback to nearest particles across the kernel.  JMB 8/29/22 */

        assert(fNorm_new != 0.0);
        fNorm_new = 1./fNorm_new;
        counter = 0;

        for(i=0;i<nSmooth;++i) {
            r2 = nnList[i].fDist2*ih2;
            KERNEL(rs,r2);
            fbweight = rs*fNorm_new;
            nnList[i].pPart->u += fbweight*dE;
            nnList[i].pPart->fNSN += fbweight*dE;
            if (nnList[i].pPart->u > smf->dBHemax) nnList[i].pPart->u = smf->dBHemax;
            if (nnList[i].pPart->fNSN > smf->dBHemax) nnList[i].pPart->fNSN = smf->dBHemax;
            counter ++;
            /* now turn off cooling */
            if(  smf->bBHTurnOffCooling) nnList[i].pPart->fTimeCoolIsOffUntil = max(nnList[i].pPart->fTimeCoolIsOffUntil,smf->dTime + nnList[i].pPart->dt);
//            printf("BHSink %d: Time %g FB Energy to %i dE %g tCoolOffUntil %g \n",p->iOrder,smf->dTime,nnList[i].pPart->iOrder,fbweight*dE,nnList[i].pPart->fTimeCoolIsOffUntil);
	    if(smf->bBHTurnOffCooling) assert(nnList[i].pPart->fTimeCoolIsOffUntil > smf->dTime);
            nnList[i].pPart->fMassForm = (FLOAT) smf->dTime;
            /* track BHFB time in MassForm  JMB 11/19/08 */
        }
    }

#endif
}
```

### postBHFeedback

```c
void postBHFeedback(PARTICLE *p1, SMF *smf)
{
#ifdef STARFORM
    /* Convert energy  back to specific quantities (per mass)
       because we are done with our conservative calculations */
    if(TYPETest(p1, TYPE_GAS) && p1->fMass != 0 && !(TYPETest(p1,TYPE_DELETED))) {
    p1->u /= p1->fMass;
    p1->fNSN /= p1->fMass;
    p1->uPred += p1->fNSN; /* now combine fNSN to uPred.  JMB 10/5/09  */
  }

#endif
}
```